# A comparison of cognitive and organizational classification of publications in the Social Sciences and Humanities:<br>additional material

This notebook is an accompaniment to the paper 'A comparison of cognitive and organizational classification of publications in the Social Sciences and Humanities', to be published in *Scientometrics*. It shows how the data were analyzed and the results were obtained.

## Loading data

First, we import the necessary modules. Next, load the data.

In [1]:
from collections import OrderedDict
from operator import itemgetter

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import seaborn as sns

from disciplines import *

%matplotlib inline
sns.set_style('ticks')

In [2]:
df = pd.read_csv('data/data.csv')

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Number of publications
len(df)

76076

Table 1:

In [4]:
df.groupby(['VABB-publicatietype', 'infogpstd']).size()

VABB-publicatietype  infogpstd
VABB-1               gp           27749
                     wos          30569
VABB-2               gp            1199
VABB-3               gp            2057
VABB-4               gp           11799
VABB-5               gp            1380
                     wos           1323
dtype: int64

Now we create three tables:
1. `fosdf`: FOS fields per publication
2. `vabbdf`: VABB disciplines per publication
3. `vabbdf_specific`: VABB disciplines per publication, excluding *Social sciences general* and *Humanities general*

In [5]:
fosdf = pivot_discipline_cols(df, [col for col in df.columns if col.startswith('FOS')])

In [6]:
vabbdf = df[vabb_discipline_names]
vabbdf_specific = df[vabb_discipline_names_specific]

Table 3:

In [7]:
fos_count = fosdf.sum(axis=1).astype(int)
vabb_count = vabbdf.sum(axis=1).astype(int)

counts = pd.DataFrame({
    'fos': fos_count,
    'vabb': vabb_count,
})

counts.groupby(['fos', 'vabb']).size().unstack().fillna(0).astype(int)

vabb,1,2,3,4,5,6,7,8
fos,,,,,,,,
1,53568,12809,2887,717,120,19,5,0
2,3804,1035,240,120,8,0,0,1
3,425,125,23,3,1,0,0,0
4,109,37,8,2,1,0,0,0
5,7,1,1,0,0,0,0,0


Use labels instead of numbers for FOS disciplines:

In [8]:
col_renamer = {num: name for num, name, _ in fos_disciplines}
fosdf = (fosdf.reindex(columns=[num for num, _, level in fos_disciplines if level != 1])
              .rename(columns=col_renamer))

## Networks

These networks show co-occurrences between VABB disciplines and between FOS fields. They can be visualized in [VOSviewer](http://www.vosviewer.com). In VOSviewer, choose `Create... > Create a map based on network data`, pick  the tab labeled `Pajek`, and fill in the locations of the network file, partiton file, and vector file.

First, we have two functions that are used to write a Pajek partition file and a Pajek vector file.

In [9]:
def write_clu(G, fname, node_clusters):
    '''Write Pajek partition file'''
    # Translate cluster names to numbers
    cluster_names = set(node_clusters.values())
    cluster_numbers = dict(zip(cluster_names, range(1, len(cluster_names) + 1)))
    # dict: node -> cluster number
    node_cluster_numbers = {k: cluster_numbers[v] for k, v in node_clusters.items()}
    
    with open(fname, 'w') as fh:
        fh.write('*Vertices {}\n'.format(len(G)))
        for n in G:
            fh.write('{}\n'.format(node_cluster_numbers[n]))
            

def write_vec(G, fname, node_sizes):
    '''Write Pajek vector file'''
    with open(fname, 'w') as fh:
        fh.write('*Vertices {}\n'.format(len(G)))
        for n in G:
            fh.write('{}\n'.format(node_sizes[n]))

In [10]:
vabbdf = vabbdf.fillna(0)
c = vabbdf.T.dot(vabbdf)
G = nx.from_pandas_adjacency(c)
nx.write_pajek(G, 'maps/vabb.net')

# Write partition
d = dict.fromkeys(vabb_discipline_names_socsci, 'Social sciences')
d.update(dict.fromkeys(vabb_discipline_names_hum, 'Humanities'))
write_clu(G, 'maps/vabb.clu', d)

# Write vector
write_vec(G, 'maps/vabb.vec', vabbdf.sum().to_dict())

FileNotFoundError: [Errno 2] No such file or directory: 'output/vabb.net'

In [ ]:
fosdf = fosdf.fillna(0)
c = fosdf.T.dot(fosdf)
G = nx.from_pandas_adjacency(c)
nx.write_pajek(G, 'maps/fos.net')

# Write partition
d  = {}
for num, name, level in fos_disciplines:
    if level == 1:
        current_top_level = name
    else:
        d[name] = current_top_level
write_clu(G, 'maps/fos.clu', d)

# Write vector
write_vec(G, 'maps/fos.vec', fosdf.sum().to_dict())

## Cooccurrences between cognitive and organizational classification

### At aggregated level

Build a DataFrame `top_fosdf` of top-level FOS field(s) per publication

In [ ]:
agg_fos = OrderedDict()
for num, name, level in fos_disciplines:
    if level == 1:
        current_list = []
        agg_fos[name] = current_list
    else:
        current_list.append(name)

top_fosdf = pd.DataFrame()
for k, v in agg_fos.items():
    top_fosdf[k] = (fosdf[v] == 1).any(axis=1).astype(int)

Build a DataFrame `top_vabbdf` of publications to top-level VABB disicplines (i.e., *Social sciences* and *Humanities*):

In [ ]:
agg_vabb = OrderedDict()
agg_vabb['Social sciences'] = vabb_discipline_names_socsci
agg_vabb['Humanities'] = vabb_discipline_names_hum

top_vabbdf = pd.DataFrame()
for k, v in agg_vabb.items():
    top_vabbdf[k] = (vabbdf[v] == 1).any(axis=1).astype(int)

Figure 3:

In [ ]:
coocc_agg = make_coocc_table(top_vabbdf, top_fosdf)
coocc_agg = normalize_coocc_table(coocc_agg)
fig, ax = plot_heatmap(coocc_agg, annot=True, figsize=(12.5, 3.5), fmt='.2f', cbar=False)
plt.yticks(rotation=0)
plt.xticks(rotation=0)
plt.tight_layout()

### At second level of fields

We start from the table without *Social sciences general* and *Humanities general*. Number of publications underlying this figure:

In [ ]:
len(vabbdf_specific[vabbdf_specific.sum(axis=1) > 0])

In [ ]:
coocc = make_coocc_table(vabbdf_specific, fosdf)
coocc_norm = normalize_coocc_table(coocc)

Figure 4:

In [ ]:
fig, ax = plot_heatmap(coocc_norm, figsize=(14, 7),
                       cmap=sns.cubehelix_palette(light=.95, as_cmap=True))

Let's see how strong the correspondence is between each VABB discipline and closest corresponding FOS field.

In [ ]:
mapping = {
    # VABB: FOS
    'Social health sciences': 'Health sciences',
    'Psychology': 'Psychology',
    'Economics & business': 'Economics and business',
    'Educational sciences': 'Educational sciences',
    'Sociology': 'Sociology',
    'Law': 'Law',
    'Criminology': 'Law',
    'Political sciences': 'Political science',
    'Communication studies': 'Media and communications',
    'History': 'History',
    'Archaeology': 'Archaeology',
    'Linguistics': 'Languages and linguistics',
    'Literature': 'Literature',
    'Philosophy': 'Philosophy and ethics',
    'Theology': 'Religion',
    'History of arts': 'Arts',
}

In [ ]:
correspondence = {k: coocc_norm.loc[k, v] for k, v in mapping.items()}
for k, v in sorted(correspondence.items(), key=itemgetter(1), reverse=True):
    print('{}: {:.1f}%'.format(k, v * 100))

## Influence of collaboration and FOS multi-assignment

Visualize the difference to the original table: increases are in red and decreases in blue. The intensity of the color indicates the strength of increase or decrease.

Figure 5:

In [ ]:
# Index of single-assignment publications
idx = df[(vabbdf.sum(axis=1) == 1) & (fosdf.sum(axis=1) == 1)].index

coocc_single_assignment = make_coocc_table(vabbdf_specific.loc[idx], fosdf.loc[idx])
coocc_single_assignment_norm = normalize_coocc_table(coocc_single_assignment)
single_assignment_diff = coocc_single_assignment_norm - coocc_norm

fig, ax = plot_heatmap(single_assignment_diff, center=0, cmap='bwr')
ax.set_xlabel('')

Single assignment at aggregate level:

In [ ]:
coocc_single_assignment_agg = make_coocc_table(top_vabbdf.loc[idx], top_fosdf.loc[idx])
normalize_coocc_table(coocc_single_assignment_agg)

Strongest increase:

In [ ]:
single_assignment_diff.max(axis=1).sort_values()[-1:]

## Gini inequality index

In [ ]:
def gini(x):
    # Source: https://stackoverflow.com/questions/39512260#39513799
    # (Warning: This is a concise implementation, but it is O(n**2)
    # in time and memory, where n = len(x).  *Don't* pass in huge
    # samples!)
    mad = np.abs(np.subtract.outer(x, x)).mean()
    rmad = mad / np.mean(x)
    
    return 0.5 * rmad

Table 4:

In [ ]:
pd.DataFrame({
    'Gini (all pubs)': coocc_norm.apply(gini, axis=1),
    'Gini (single-assignment)': coocc_single_assignment_norm.apply(gini, axis=1),
}).sort_values(ascending=False, by='Gini (all pubs)')


Gini index for aggregated VABB disciplines *Social sciences* and *Humanities*:

In [ ]:
coocc_agg.apply(gini, axis=1)